In [1]:
# Check if running in Google Colab
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    # Define the base path for your data directory for Colab
    basePath = '/content/drive/MyDrive'

else:
    # Running locally or in a different environment
    basePath = '..'

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display

In [3]:
file_path = f'data/processed/cleaned_data1.csv'
df = pd.read_csv(f'{basePath}/{file_path}')

In [4]:
# Define columns to group by
group_cols = ['COUNTRY', 'DATE']

# Identify numeric columns to aggregate (excluding the grouping columns)
numeric_cols_for_agg = df.select_dtypes(include=np.number).columns.tolist()
numeric_cols_for_agg = [col for col in numeric_cols_for_agg if col not in group_cols]

# Group by COUNTRY and DATE and calculate the mean of numeric columns
df_grouped = df.groupby(group_cols)[numeric_cols_for_agg].mean().reset_index()

print("Grouped data by Country and Year (re-calculated):")
display(df_grouped.head())

Grouped data by Country and Year (re-calculated):


,COUNTRY,DATE,LATITUDE,LONGITUDE,ELEVATION,CDSD,CLDD,DP01,DP10,DT00,...,FZF0,FZF5,FZF6,HDSD,HDSD_ATTRIBUTES,HTDD,PRCP,TAVG,TMAX,TMIN
0,Albania,1981,41.4150,19.7210,38.4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.8,NaN
1,Albania,2016,41.4150,19.7210,38.4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.1,NaN
2,Algeria,1892,30.5667,2.8667,397.0,NaN,NaN,15.0,8.0,0.0,...,-0.5,-1.0,NaN,NaN,NaN,NaN,65.7,NaN,NaN,13.9
3,Algeria,1893,30.5667,2.8667,397.0,NaN,NaN,7.0,4.0,0.0,...,0.0,0.0,-4.0,NaN,NaN,NaN,25.4,NaN,NaN,13.4
4,Algeria,1895,30.5667,2.8667,397.0,NaN,NaN,6.0,0.0,0.0,...,NaN,0.0,-2.6,NaN,NaN,NaN,6.6,NaN,NaN,13.7


In [ ]:
# Define the time periods (using the same periods as before)
start_year_period1 = 2000
end_year_period1 = 2012
start_year_period2 = 2013
end_year_period2 = 2025 

# Filter df_grouped for the two time periods
df_period1 = df_grouped[(df_grouped['DATE'] >= start_year_period1) & (df_grouped['DATE'] <= end_year_period1)].copy()

df_period2 = df_grouped[(df_grouped['DATE'] >= start_year_period2) & (df_grouped['DATE'] <= end_year_period2)].copy()

In [ ]:
group_cols = ['COUNTRY', 'DATE']

# We need to be careful about which columns to include. Let's exclude the ones we know are identifiers or attributes.
exclude_cols = group_cols + ['STATION', 'STATION_NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION']

attribute_cols = [col for col in df_grouped.columns if '_ATTRIBUTES' in col]
exclude_cols.extend(attribute_cols)

numeric_cols_to_analyze = [col for col in df_grouped.select_dtypes(include=np.number).columns if col not in exclude_cols]

In [ ]:
# Dictionary to store the change Dataframes (clearing previous results if any)
change_dataframes = {}

for col in numeric_cols_to_analyze:
    print(f"\nCalculating percentage change for: {col}")

    avg_period1 = df_period1.groupby('COUNTRY')[col].mean().reset_index()
    avg_period1.rename(columns={col: f'Avg_{col}_Period1'}, inplace=True)

    avg_period2 = df_period2.groupby('COUNTRY')[col].mean().reset_index()
    avg_period2.rename(columns={col: f'Avg_{col}_Period2'}, inplace=True)

    # Merge the two dataframes on the 'COUNTRY' column
    # Use outer merge to keep all countries from both periods
    change_df = pd.merge(avg_period1, avg_period2, on='COUNTRY', how='outer')

    epsilon = 1e-9
    change_df[f'{col}_Change_Percent'] = ((change_df[f'Avg_{col}_Period2'] - change_df[f'Avg_{col}_Period1']) / (change_df[f'Avg_{col}_Period1'] + epsilon)) * 100

    change_dataframes[f'{col}_change'] = change_df

    print(f"Top 5 countries by percentage change in {col}:")
    display(change_df.sort_values(by=f'{col}_Change_Percent', ascending=False).head())


Calculating percentage change for: CDSD
Top 5 countries by percentage change in CDSD:


,COUNTRY,Avg_CDSD_Period1,Avg_CDSD_Period2,CDSD_Change_Percent
112,Svalbard [Norway],0.000000,0.025000,2.500000e+09
114,Switzerland,0.000000,0.010000,1.000000e+09
56,Ireland,1.200000,2.454545,1.045455e+02
90,Norway,3.932436,7.322222,8.620068e+01
107,Slovakia,53.569231,78.000000,4.560597e+01



Calculating percentage change for: CLDD
Top 5 countries by percentage change in CLDD:


,COUNTRY,Avg_CLDD_Period1,Avg_CLDD_Period2,CLDD_Change_Percent
112,Svalbard [Norway],0.000000,0.025000,2.500000e+09
114,Switzerland,0.000000,0.010000,1.000000e+09
56,Ireland,1.200000,2.454545,1.045455e+02
90,Norway,3.932436,7.322222,8.620068e+01
107,Slovakia,53.569231,78.000000,4.560597e+01



Calculating percentage change for: DP01
Top 5 countries by percentage change in DP01:


...


Calculating percentage change for: TMAX
Top 5 countries by percentage change in TMAX:


,COUNTRY,Avg_TMAX_Period1,Avg_TMAX_Period2,TMAX_Change_Percent
48,Greenland [Denmark],-0.534722,-2.868333,436.415585
3,Antarctica,-4.220000,-8.771429,107.853758
101,Romania,7.156667,12.762500,78.330228
102,Russia,2.613253,4.263974,63.167259
114,Switzerland,2.161538,2.922727,35.215141



Calculating percentage change for: TMIN
Top 5 countries by percentage change in TMIN:


,COUNTRY,Avg_TMIN_Period1,Avg_TMIN_Period2,TMIN_Change_Percent
118,Turkey,9.726190,16.166667,66.217870
3,Antarctica,-9.624000,-15.925000,65.471737
107,Slovakia,1.569231,2.475000,57.720588
101,Romania,2.465385,3.604545,46.206212
7,Austria,2.115385,2.903333,37.248485



Percentage change dataframes stored in 'change_dataframes' dictionary.


In [8]:
df_grouped.to_csv(f'{basePath}/data/processed/grouped_data2.csv', index=False)

In [9]:
#Note: the change dataframes will be used in th enext file to generate the graphs/charts so saving it

In [10]:
import pickle
import os

# Define the directory to save the file
save_dir = f'{basePath}/data/processed'
os.makedirs(save_dir, exist_ok=True)

# Define the file path
file_path = os.path.join(save_dir, 'change_dataframes.pkl')

# Save the dictionary using pickle
with open(file_path, 'wb') as f:
    pickle.dump(change_dataframes, f)

print(f"change_dataframes dictionary saved to {file_path}")

change_dataframes dictionary saved to ../data/processed/change_dataframes.pkl
